In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np
from bs4 import BeautifulSoup
import re

In [2]:
#Reading tables from website
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dataframe_list = pd.read_html(url, flavor='bs4')

In [3]:
#Data is on first table
table = dataframe_list[0]

In [4]:
#Converting table to array
data = table.to_numpy()

In [5]:
#Converting to 1D array
data = data.reshape(data.shape[0]*data.shape[1],)

In [6]:
data_df = pd.DataFrame(data)

In [7]:
# all info in 'all' column
data_df.columns = ['all']

In [8]:
# "PostCode" extracted
data_df['PostalCode'] = data_df['all'].str.extract('(.{3})',expand=False)

In [9]:
data_df.head()

,all,PostalCode
0,M1ANot assigned,M1A
1,M2ANot assigned,M2A
2,M3ANorth York(Parkwoods),M3A
3,M4ANorth York(Victoria Village),M4A
4,M5ADowntown Toronto(Regent Park / Harbourfront),M5A


In [10]:
#all info except 'PostalCode' in in temp 'rest' column
data_df['rest']=data_df['all'].str.replace(r'^(.{3})','')

<ipython-input-10-b6930228a476>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  data_df['rest']=data_df['all'].str.replace(r'^(.{3})','')


In [11]:
data_df.drop(columns=['all'],inplace=True)

In [12]:
data_df.shape

(180, 2)

In [13]:
# records with no 'Borough' removed
data_df = data_df[data_df['rest'] != 'Not assigned']

In [14]:
data_df.shape

(103, 2)

In [15]:
# "Borough" extracted
data_df['Borough'] = data_df['rest'].str.extract('(.+?)\(.+',expand=False)

In [16]:
data_df.head()

,PostalCode,rest,Borough
2,M3A,North York(Parkwoods),North York
3,M4A,North York(Victoria Village),North York
4,M5A,Downtown Toronto(Regent Park / Harbourfront),Downtown Toronto
5,M6A,North York(Lawrence Manor / Lawrence Heights),North York
6,M7A,Queen's Park(Ontario Provincial Government),Queen's Park


In [17]:
# 'Borough' data removed from 'rest'
data_df['rest'] = data_df['rest'].str.replace('^[^(]+','')

<ipython-input-17-2508a9661765>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  data_df['rest'] = data_df['rest'].str.replace('^[^(]+','')


In [18]:
# "Neighborhood" extracted
data_df['Neighborhood'] = data_df['rest'].str.extract('\((.+)\).{0,}',expand=False)

In [19]:
data_df.head()

,PostalCode,rest,Borough,Neighborhood
2,M3A,(Parkwoods),North York,Parkwoods
3,M4A,(Victoria Village),North York,Victoria Village
4,M5A,(Regent Park / Harbourfront),Downtown Toronto,Regent Park / Harbourfront
5,M6A,(Lawrence Manor / Lawrence Heights),North York,Lawrence Manor / Lawrence Heights
6,M7A,(Ontario Provincial Government),Queen's Park,Ontario Provincial Government


In [20]:
#'rest' column dropped
data_df.drop(columns=['rest'],inplace=True)

In [21]:
# index reset
data_df.reset_index(drop=True,inplace=True)

In [22]:
data_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Queen's Park,Ontario Provincial Government


In [23]:
data_df.shape

(103, 3)

In [24]:
coordinates_df = pd.read_csv('Geospatial_Coordinates.csv')

In [25]:
coordinates_df.shape

(103, 3)

In [26]:
data_df.sort_values(by=['PostalCode'],inplace=True)

In [27]:
coordinates_df.sort_values(by=['Postal Code'],inplace=True)

In [28]:
data_df.reset_index(inplace=True,drop=True)

In [29]:
coordinates_df.reset_index(inplace=True,drop=True)

In [30]:
data_df

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,Kingsview Village / St. Phillips / Martin Grov...
101,M9V,Etobicoke,South Steeles / Silverstone / Humbergate / Jam...


In [31]:
coordinates_df

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [32]:
data_df['Latitude'] = coordinates_df['Latitude']

In [33]:
data_df['Longitude'] = coordinates_df['Longitude']

In [34]:
data_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
